In [1]:
# !python -m venv C:\GreenGIS\NearMissDash
# !C:\GreenGIS\NearMissDash\Scripts\activate.bat
# !PS C:\GreenGIS\NearMissDash\Scripts\Activate.ps1

In [2]:
# pip intall leaflet-image

In [3]:
#import packages
from flask import Flask, render_template, request, jsonify 
from flask_json import FlaskJSON, JsonError, json_response
from flask_mail import Mail, Message 
import folium
from IPython.display import HTML, display
import json
import pandas as pd
import numpy as np
import os
from collections import Counter
import io
import smtplib
from email.mime.text import MIMEText
from waitress import serve
import requests
import sqlite3
import urllib.request

#name app
app = Flask(__name__)
app.config['SECRET_KEY'] = 'greenstreets'

#setup JSONBin.io 
JSONBIN_ACCESS_KEY = '$2a$10$J23yWei2a5JsxL9JkiWOXuZsgZ.qR/GRj74Jre.4i/Te3XjaT0A2y'
JSONBIN_API_URL = f'https://api.jsonbin.io/v3/b/66db5907acd3cb34a87f7d42' # BIN
JSONBIN_API_URL2 = f'https://api.jsonbin.io/v3/b/66db2f75ad19ca34f8a0f264' # EVENTS

#tell flask to read home page
@app.route('/')
def index(): 
    return render_template('apphome.html')

#tell flask to read defs page
@app.route('/defs')
def defs(): 
    return render_template('defs.html')

#tell flask to read dashboard page
@app.route('/dashboard')
def dashboard():
    # Read content of plot1.html
    with open("plot1.html", "r") as f:
        plot1_html = f.read()
    
    # Read content of plot2.html
    with open("plot2.html", "r") as f:
        plot2_html = f.read()
    
    # Pass both plots to the template
    return render_template("dash2test.html", plot1_html=plot1_html, plot2_html=plot2_html)



#Tell Flask to create a record
@app.route('/add-feature', methods=['POST'])
def add_feature():
    new_feature = request.json  # Get the new feature from the request body
    
    # Fetch the current data from JSONBin
    headers = {
        'X-Master-Key': JSONBIN_ACCESS_KEY,
    }
    response = requests.get(JSONBIN_API_URL, headers=headers)

    if response.status_code != 200:
        return jsonify({'error': 'Failed to fetch data from JSONBin.io'}), response.status_code

    # Add the new feature to the existing data
    data = response.json()['record']
    data['features'].append(new_feature)

    # Update the JSONBin with the new data
    update_response = requests.put(JSONBIN_API_URL, headers=headers, json=data)

    if update_response.status_code == 200:
        return jsonify(update_response.json()), 200
    else:
        return jsonify({'error': 'Failed to update data in JSONBin.io'}), update_response.status_code

#Tell Flask how to read bin - map on dashboard page
@app.route('/get-map-data',  methods=['GET'])
def get_map_data():
    headers = {
        'X-Master-Key': JSONBIN_ACCESS_KEY,
    }
    response = requests.get(JSONBIN_API_URL, headers=headers)

    if response.status_code == 200:
        return jsonify(response.json())
    else:
        return jsonify({"error": "Failed to fetch data"}), 500



#tell flask run app
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Feb/2025 14:32:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2025 14:32:25] "GET /static/bootstrap-slider/css/bootstrap-slider.min.css HTTP/1.1" 404 -
127.0.0.1 - - [26/Feb/2025 14:32:25] "GET /docs/5.3/dist/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [26/Feb/2025 14:32:25] "GET /static/bootstrap/css/bootstrap-theme.min.css HTTP/1.1" 404 -
127.0.0.1 - - [26/Feb/2025 14:32:25] "GET /static/nearmiss1.png HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2025 14:32:25] "GET /static/stylesheet.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2025 14:32:25] "GET /static/PlanRVA_White.png HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2025 14:32:25] "GET /favicon.ico HTTP/1.1" 404 -


: 